In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import math

In [2]:
df=pd.read_csv("../facial_expressions/data/legend.csv").drop("user.id",axis=1)

In [3]:
IMG_PATH="../facial_expressions/images/"

In [4]:
emotions=["neutral", "anger", "disgust", "fear", "happy", "sadness"]

In [5]:
df["emotion"]=df["emotion"].map(lambda x: x.lower())

In [6]:
df=df.replace(to_replace ="contempt", value ="disgust")
df=df.replace(to_replace ="surprise", value ="happy")
df=df.replace(to_replace ="happiness", value ="happy")
df=df.replace(to_replace ="sad", value ="sadness")

In [7]:
inds=[]
for em in df["emotion"]:
    inds.append(emotions.index(em))
df["index"]=inds

In [8]:
df

,image,emotion,index
0,facial-expressions_2868588k.jpg,anger,1
1,facial-expressions_2868585k.jpg,happy,4
2,facial-expressions_2868584k.jpg,disgust,2
3,facial-expressions_2868582k.jpg,fear,3
4,Aaron_Eckhart_0001.jpg,neutral,0
...,...,...,...
14185,Tim_Pawlenty_0001.jpg,neutral,0
14186,Tim_Robbins_0001.jpg,neutral,0
14187,Tim_Robbins_0002.jpg,neutral,0
14188,Tim_Robbins_0003.jpg,neutral,0


In [9]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [10]:
face_dim=64

In [40]:
def ret_keypoints(imname):
    try:
        image = cv2.imread(IMG_PATH+imname)
    except:
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim)) 
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
#     boxes = [(y,x+w,y+h,x) for (x,y,w,h) in faces]
    shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(float(shape.part(i).x))
        ylist.append(float(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim + 1)/2

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [13]:
model=Sequential()
model.add(Dense(1024,activation='relu', input_shape=(68*2,)))
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(len(emotions),activation='relu'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              140288    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 774       
Total params: 830,086
Trainable params: 830,086
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
df.iloc[0:10]

,image,emotion,index
0,facial-expressions_2868588k.jpg,anger,1
1,facial-expressions_2868585k.jpg,happy,4
2,facial-expressions_2868584k.jpg,disgust,2
3,facial-expressions_2868582k.jpg,fear,3
4,Aaron_Eckhart_0001.jpg,neutral,0
5,Aaron_Guiel_0001.jpg,happy,4
6,Aaron_Patterson_0001.jpg,neutral,0
7,Aaron_Peirsol_0001.jpg,happy,4
8,Aaron_Peirsol_0002.jpg,happy,4
9,Aaron_Peirsol_0003.jpg,happy,4


In [45]:
BATCH_SIZE=64
GCT=0

In [46]:
def get_batch():
    global GCT
    X_inp=np.empty((BATCH_SIZE,68*2),dtype=np.float32)
    y_inp=np.zeros((BATCH_SIZE,len(emotions)),dtype=np.float32)
    ct=0
    while ct<BATCH_SIZE:
        row=df.iloc[GCT]
        GCT+=1
        landmarks=ret_keypoints(row["image"])
        if landmarks is None:
            continue
        else:
            X_inp[ct]=landmarks
            y_inp[ct][row["index"]]=1
            ct+=1
    return X_inp,y_inp

In [81]:
def trainer():
    for i in range(len(df)//BATCH_SIZE + 1):
        X_inp,y_inp=get_batch()
        model.train_on_batch(X_inp,y_inp)